In [1]:
%pip install -U -q 'google-generativeai>=0.8.3'

In [2]:
import google.generativeai as genai
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
GOOGLE_API_KEY = "<key>"
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
for model in genai.list_models():
    if "createTunedModel" in model.supported_generation_methods:
        print(model.name)

In [ ]:
for model_info in genai.list_tuned_models():
    print(model_info.name)

In [ ]:
df = pd.read_csv('Phishing_Email.csv', index_col=0)
df=df.dropna()
train, test = train_test_split(df, test_size=0.2)
test

In [19]:
from google.api_core import retry

baseline_model = genai.GenerativeModel("gemini-1.5-flash-001")

system_instruct = """
You are a classification service. You will be passed input that represents
an email and you must respond with whether the email is safe or if it is a Phishing Email. The output should be either 'Safe Email' or 'Phishinh Email'.
"""

instructed_model = genai.GenerativeModel("gemini-1.5-flash-001", system_instruction=system_instruct)

retry_policy = {"retry": retry.Retry(predicate=retry.if_transient_error)}

def predict_label(post: str) -> str:
    response = instructed_model.generate_content(post, request_options=retry_policy)
    rc = response.candidates[0]

    if rc.finish_reason.name != "STOP":
        return "(error)"
    else:
        return response.text.strip()

In [ ]:
from tqdm.rich import tqdm
tqdm.pandas()

df_baseline_eval = test.sample(64)

df_baseline_eval['Prediction'] = df_baseline_eval['Email Text'].progress_apply(predict_label)

accuracy = (df_baseline_eval["Email Type"] == df_baseline_eval["Prediction"]).sum() / len(df_baseline_eval)
print(f"Accuracy: {accuracy:.2%}")

In [ ]:
TP = ((df_baseline_eval["Email Type"] == "Phishing Email") & (df_baseline_eval["Prediction"] == "Phishing Email")).sum()
TN = ((df_baseline_eval["Email Type"] == "Safe Email") & (df_baseline_eval["Prediction"] == "Safe Email")).sum()
FP = ((df_baseline_eval["Email Type"] == "Safe Email") & (df_baseline_eval["Prediction"] == "Phishing Email")).sum()
FN = ((df_baseline_eval["Email Type"] == "Phishing Email") & (df_baseline_eval["Prediction"] == "Safe Email")).sum()

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)
accuracy = (df_baseline_eval["Email Type"] == df_baseline_eval["Prediction"]).sum() / len(df_baseline_eval)

print(f"Accuracy: {accuracy:.2%}")
print(f"precision: {precision:.2%}")
print(f"recall: {recall:.2%}")
print(f"f1_score: {f1_score}")

In [ ]:
from collections.abc import Iterable
import random

model_id = f"phishing-email-classifier-{random.randint(10000, 99999)}"

tuning_operation = genai.create_tuned_model(
    "models/gemini-1.5-flash-001-tuning",
    training_data=train.sample(512),
    input_key="Email Text",
    output_key="Email Type",
    id=model_id,
    display_name="Phishing Email Classification Model",
    batch_size=4,
    epoch_count=20,
    learning_rate=0.001,
)

print(model_id)

In [ ]:
import time
import seaborn as sns


while (tuned_model := genai.get_tuned_model(f"tunedModels/{model_id}")).state.name != 'ACTIVE':

    if tuned_model.tuning_task.snapshots:
        snapshots = pd.DataFrame(tuned_model.tuning_task.snapshots)
        sns.lineplot(data=snapshots, x="step", y="mean_loss")

    print(tuned_model.state)

    time.sleep(60)

print(f"Done! The model is {tuned_model.state.name}")

In [ ]:
your_model = genai.GenerativeModel(f"tunedModels/{model_id}")

import vertexai

from vertexai.generative_models import (
    GenerativeModel,
    HarmCategory,
    HarmBlockThreshold,
    Part,
    SafetySetting,
)

safety_config = {
            'HATE': 'BLOCK_NONE',
            'HARASSMENT': 'BLOCK_NONE',
            'SEXUAL' : 'BLOCK_NONE',
            'DANGEROUS' : 'BLOCK_NONE'
}

def classify_email(text: str) -> str:
    response = your_model.generate_content(text, request_options=retry_policy, safety_settings=safety_config)
    rc = response.candidates[0]

    if rc.finish_reason.name != "STOP":
        return "(error)"
    else:
        return rc.content.parts[0].text

df_model_eval = test.sample(64)


df_model_eval["Prediction"] = df_model_eval["Email Text"].progress_apply(classify_email)

accuracy = (df_model_eval["Email Type"] == df_model_eval["Prediction"]).sum() / len(df_model_eval)
print(f"Accuracy: {accuracy:.2%}")

In [ ]:
TP = ((df_model_eval["Email Type"] == "Phishing Email") & (df_model_eval["Prediction"] == "Phishing Email")).sum()
TN = ((df_model_eval["Email Type"] == "Safe Email") & (df_model_eval["Prediction"] == "Safe Email")).sum()
FP = ((df_model_eval["Email Type"] == "Safe Email") & (df_model_eval["Prediction"] == "Phishing Email")).sum()
FN = ((df_model_eval["Email Type"] == "Phishing Email") & (df_model_eval["Prediction"] == "Safe Email")).sum()

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)
accuracy = (df_model_eval["Email Type"] == df_model_eval["Prediction"]).sum() / len(df_model_eval)

print(f"Accuracy: {accuracy:.2%}")
print(f"precision: {precision:.2%}")
print(f"recall: {recall:.2%}")
print(f"f1_score: {f1_score}")